In [168]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
## for correlation matrices
import seaborn as sns
%matplotlib inline
## for linear models
import statsmodels.api as sm
from pandas.plotting import scatter_matrix
from sklearn.metrics import roc_curve, auc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from yellowbrick.classifier import ConfusionMatrix


from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing

from imblearn.over_sampling import SMOTE

from sklearn.metrics import classification_report
%run ../pyfiles/data_cleaning.py
%run ../pyfiles/unmetDemand.py

In [114]:
df = pd.read_csv("../data/deepsolar_tract.csv", encoding = "utf-8")


In [115]:
df = drop_redundant_columns(df)

In [116]:
df = create_has_tiles_target_column(df)

In [117]:
df.columns

Index(['tile_count', 'fips', 'average_household_income', 'education_bachelor',
       'education_college', 'education_doctoral',
       'education_high_school_graduate', 'education_less_than_high_school',
       'education_master', 'education_population',
       ...
       'incentive_residential_state_level',
       'incentive_nonresidential_state_level', 'net_metering', 'feedin_tariff',
       'cooperate_tax', 'property_tax', 'sales_tax', 'rebate',
       'avg_electricity_retail_rate', 'has_tiles'],
      dtype='object', length=132)

In [118]:
df = df.dropna()
df = df.drop('tile_count', axis = 1)

In [119]:
# Define X and y
y = df['has_tiles']
X = df.drop('has_tiles', axis = 1)
# Split the data into training and test sets
# from sklearn.model_selection import train_test_split

#ADD STRATIFIED HERE

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
#print(y_train.value_counts(),'\n\n', y_test.value_counts())

# SMOTE
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)
print(y_train.value_counts(),'\n\n', y_test.value_counts())

1    31209
0    31209
Name: has_tiles, dtype: int64 

 1    10392
0     3133
Name: has_tiles, dtype: int64


In [120]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [121]:
parameters = {'penalty':['l1', 'l2'], 'C':[.001, .005, .0005, .0009]}
#clf = classifier
clf = GridSearchCV(LogisticRegression(solver='saga'), parameters, cv = 5)
clf.fit(X_train, y_train)

C:\Users\BudBe\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\BudBe\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\BudBe\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\BudBe\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\BudBe\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='saga',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.001, 0.005, 0.0005, 0.0009],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [122]:
clf.best_params_

{'C': 0.005, 'penalty': 'l2'}

In [126]:
best_clf=clf.best_estimator_

In [160]:
falsePositives = isFalsePositive(df, X_test, y_test, best_clf)

In [170]:
buy = isOpportunityZone(falsePositives, scaler = scaler)

In [172]:
buy

,key_0,fips,average_household_income,education_bachelor,education_college,education_doctoral,education_high_school_graduate,education_less_than_high_school,education_master,education_population,...,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate,State,County,Census_Tract_Number,Tract_Type,ACS_Data_Source
0,27047180600,2.704718e+10,44522.559294,196.0,1037.0,8.0,946.0,541.0,48.0,2833.0,...,0.0,25.0,12.0,0.0,9.46,Minnesota,Freeborn,27047180600,Low-Income Community,2011-2015
1,33017080203,3.301708e+10,66078.894472,243.0,95.0,75.0,104.0,13.0,237.0,789.0,...,0.0,41.0,0.0,8.0,15.08,New Hampshire,Strafford,33017080203,Low-Income Community,2011-2015
2,51670820600,5.167082e+10,46826.293996,238.0,1074.0,0.0,1122.0,799.0,83.0,3325.0,...,0.0,40.0,0.0,0.0,9.12,Virginia,Hopewell,51670820600,Low-Income Community,2011-2015
3,36063020500,3.606302e+10,25052.004111,110.0,326.0,22.0,605.0,438.0,24.0,1536.0,...,0.0,40.0,12.0,7.0,15.32,New York,Niagara,36063020500,Low-Income Community,2011-2015
4,48439106201,4.843911e+10,37826.919901,42.0,715.0,0.0,972.0,826.0,24.0,2586.0,...,0.0,36.0,0.0,0.0,8.66,Texas,Tarrant,48439106201,Low-Income Community,2011-2015
5,48061012102,4.806101e+10,33427.402597,71.0,635.0,6.0,822.0,1498.0,15.0,3058.0,...,0.0,36.0,0.0,0.0,8.66,Texas,Cameron,48061012102,Low-Income Community,2011-2015
6,44007015200,4.400702e+10,24558.085382,166.0,340.0,32.0,434.0,855.0,70.0,1907.0,...,16.0,37.0,12.0,0.0,15.02,Rhode Island,Providence,44007015200,Low-Income Community,2011-2015
7,6029005507,6.029006e+09,63188.167939,445.0,1766.0,0.0,1474.0,558.0,221.0,4479.0,...,0.0,38.0,0.0,10.0,14.72,California,Kern,6029005507,Low-Income Community,2011-2015
8,26125175000,2.612518e+10,49245.068394,209.0,446.0,0.0,976.0,346.0,98.0,2075.0,...,0.0,0.0,0.0,0.0,11.00,Michigan,Oakland,26125175000,Low-Income Community,2011-2015
9,37179020602,3.717902e+10,53136.660389,326.0,715.0,29.0,885.0,1041.0,95.0,3115.0,...,0.0,9.0,0.0,0.0,9.26,North Carolina,Union,37179020602,Low-Income Community,2011-2015


In [ ]:
falsePositives.columns